# Financial Analysis with the `stock_analysis` package

The [`stock_analysis`](https://github.com/stefmolin/stock-analysis) package can be used for basic technical analysis of financial assets. Here is a breakdown of the classes and their purposes:

| Purpose | Class(es) | Module |
| --- | --- | --- |
| Collecting the data from various sources | `StockReader` | `stock_reader.py` |
| Visualizing the data | `Visualizer`, `StockVisualizer`, `AssetGroupVisualizer` | `stock_visualizer.py` |
| Calculating financial metrics | `StockAnalyzer`, `AssetGroupAnalyzer` | `stock_analyzer.py` |
| Modeling the data | `StockModeler` | `stock_modeler.py` |


## Data Collection

The `StockReader` class in the `stock_analysis` package makes it easy to collect financial data, like stock market or bitcoin prices. We simply have to create an instance by specifying the date range. Then, we can grab the data:

In [ ]:
import stock_analysis

reader = stock_analysis.StockReader('2019-01-01', '2020-12-31')

# Get faang data
fb, aapl, amzn, nflx, goog = (
    reader.get_ticker_data(ticker)
    for ticker in ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG']
)

# Get S&P 500 data
sp = reader.get_index_data('S&P 500')

# Get bitcoin data in USD
bitcoin = reader.get_bitcoin_data('USD')

## EDA
We can group our assets together in dataframes in order to perform EDA:

In [ ]:
from stock_analysis.utils import group_stocks, describe_group

faang = group_stocks(
    {
        'Facebook': fb, 
        'Apple': aapl, 
        'Amazon': amzn, 
        'Netflix': nflx, 
        'Google': goog
    }
)

faang_sp = group_stocks(
    {
        'Facebook': fb, 
        'Apple': aapl, 
        'Amazon': amzn, 
        'Netflix': nflx, 
        'Google': goog,
        'S&P 500': sp
    }
)

all_assets = group_stocks(
    {
        'Bitcoin': bitcoin,
        'S&P 500': sp,
        'Facebook': fb, 
        'Apple': aapl, 
        'Amazon': amzn, 
        'Netflix': nflx, 
        'Google': goog
    }
)


This makes it easy to describe them all at once:

In [ ]:
describe_group(all_assets).loc['close',]

### Visualizations
The `StockVisualizer` class makes it easy to create a variety of visualizations for financial analysis. We simply pass in the dataframe of the asset we want to visualize:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

netflix_viz = stock_analysis.StockVisualizer(nflx)

We can use the moving average to smooth out the trend for the stock. Notice that, in the shaded region, the 90-day moving average is acting like a ceiling for the stock price:

In [ ]:
ax = netflix_viz.moving_average('close', ['30D', '90D'])

netflix_viz.shade_region(
    ax, 
    x=('2019-10-01', '2020-07-01'), 
    color='blue', 
    alpha=0.1
)

ax.set(
    title='Netflix Closing Price', 
    ylabel='price ($)'
)

Sometimes exponential smoothing makes more sense because we can place higher importance on more recent values:

In [ ]:
ax = netflix_viz.exp_smoothing('close', [30, 90])

netflix_viz.shade_region(
    ax, 
    x=('2020-04-01', '2020-10-01'), 
    color='blue', 
    alpha=0.1
)

ax.set(
    title='Netflix Closing Price', 
    ylabel='price ($)'
)

---

**OPTIONAL SECTION**: Interactive Visualization of Moving Averages

---

We will now use widgets to create an interactive plot to see what different windows do to the moving average and exponentially smoothed moving average. Using the widget backend requires `ipympl`, which we installed back in chapter 1. However, depending on your Jupyter setup and machine you may also need to install `node.js` and with run a few commands from the command line. Try to run the cells below, and if that doesn't work follow the installation instructions [here](https://github.com/matplotlib/jupyter-matplotlib). Make sure to restart the kernel when you are set up.

First, we run this magic to enable the widgets back-end:

In [ ]:
%matplotlib widget

Next, we run the code to generate the interactive visualization. This uses the `interact()` function from `ipywidgets`:

In [ ]:
%matplotlib widget
from ipywidgets import interact


def moving_averages(viz):
    def plot_viz(period=20, exponentially_smoothed=False):
        if exponentially_smoothed:
            ax = netflix_viz.exp_smoothing('close', period)
        else:
            ax = netflix_viz.moving_average('close', str(period) + 'D')
        ax.set(
            title='Netflix Closing Price', 
            ylabel='price ($)'
        )
    return plot_viz


interact(
    moving_averages(netflix_viz), 
    period=(5, 200, 5), 
    exponentially_smoothed=False
)


*More information on the `interact()` function can be found [here](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html).*

---

When finished with the interactive visualization, be sure to switch back to `inline` plots for the rest of the notebook:

In [ ]:
%matplotlib inline

---

**END OPTIONAL SECTION**

---

Let's take a look at the effect of after-hours trading:

In [ ]:
netflix_viz.after_hours_trades()

We can use a **candlestick plot** to study the OHLC data. More information on reading and interpreting this can be found [here](https://www.investopedia.com/trading/candlestick-charting-what-is-it/).

In [ ]:
netflix_viz.candlestick(
    resample='2W', 
    volume=True, 
    xrotation=90, 
    datetime_format='%Y-%b -'
)

The `mplfinance` package uses it's own stylesheets for plotting, so let's restore the defaults:

In [ ]:
import matplotlib as mpl
mpl.rcdefaults()
%matplotlib inline

We can use a joint plot to see how Netflix compared to the S&P 500:

In [ ]:
netflix_viz.jointplot(sp, 'close')

We can also use the `correlation_heatmap()` function to look at the correlation of daily percent change. Here, we compare with Amazon:

In [ ]:
netflix_viz.correlation_heatmap(amzn)

We can use the `fill_between_other()` method to compare Netflix's stock price over time to another asset. Here, we look at Tesla:

In [ ]:
tsla = reader.get_ticker_data('TSLA')

change_date = (tsla.close > nflx.close).idxmax()

ax = netflix_viz.fill_between_other(tsla)
netflix_viz.add_reference_line(
    ax, 
    x=change_date, 
    color='k', 
    linestyle=':', 
    label=f'TSLA > NFLX {change_date:%Y-%m-%d}',
    alpha=0.5
)


If we want to visualize groups of assets, we use the `AssetGroupVisualizer` class. Let's look at the correlations between the assets using percentage change:

In [ ]:
all_assets_viz = stock_analysis.AssetGroupVisualizer(all_assets)

all_assets_viz.heatmap()

Bitcoin is on a much larger scale so we have to use subplots to compare it to the stock market:

In [ ]:
faang_sp_viz = stock_analysis.AssetGroupVisualizer(faang_sp)
bitcoin_viz = stock_analysis.StockVisualizer(bitcoin)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

faang_sp_viz.evolution_over_time(
    'close', 
    ax=axes[0], 
    style=faang_sp_viz.group_by
)
bitcoin_viz.evolution_over_time(
    'close', 
    ax=axes[1], 
    label='Bitcoin'
)


### Additional Plots (not in text)
We can view the effect that after-hours trading had on each of the FAANG stocks:

In [ ]:
faang_viz = stock_analysis.AssetGroupVisualizer(faang)
faang_viz.after_hours_trades()

We can look at what the exponentially smoothed moving averages look like for all FAANG stocks as well:

In [ ]:
faang_viz.exp_smoothing('close', 20)

With short windows, both these smoothing methods perform similarly:

In [ ]:
faang_viz.moving_average('close', '20D')

The distributions of closing price across the FAANG stocks are quite different:

In [ ]:
faang_viz.histogram('close')

Viewing box plots we can easily see that Google and Amazon are on a much larger scale:

In [ ]:
faang_viz.boxplot('close')
plt.yscale('log')

Correlations between the stocks are weak:

In [ ]:
faang_viz.heatmap()

We can view this information as scatter plots as well:

In [ ]:
faang_viz.pairplot()

## Technical Analysis
Technical analysis involves looking at metrics to understand the peformance of an asset.
### Single Asset
We can use the `StockAnalyzer` class to calculate metrics for us:

In [ ]:
nflx_analyzer = stock_analysis.StockAnalyzer(nflx)

One such metric is annualized volatility:

In [ ]:
nflx_analyzer.calc_annualized_volatility()

### Asset Groups
When working with groups, we use the `AssetGroupAnalyzer` class:

In [ ]:
all_assets_analyzer = stock_analysis.AssetGroupAnalyzer(all_assets)

#### Coefficient of Variation
We can check how large the standard deviation is in terms of the mean of the stock price as another way to look at volatility. This shows that bitcoin is the most volatile:

In [ ]:
all_assets_analyzer.analyze('calc_cv')

#### Annualized Volatility

In [ ]:
all_assets_analyzer.analyze('calc_annualized_volatility')

#### Bull Market
We can check if the assets are in a bull market, meaning their value increased $\geq$ 20% in the last 2 months:

In [ ]:
all_assets_analyzer.analyze('is_bull_market')

#### Bear Market
We can check if the assets are in a bear market, meaning their value dropped $\geq$ 20% in the last 2 months:

In [ ]:
all_assets_analyzer.analyze('is_bear_market')

#### Beta
Beta tells us how an asset's performance correlates to a market index. A beta of 1 with the S&P 500 means the asset moves with the S&P 500. Notice bitcoin is below one because it is a different type of asset.

In [ ]:
all_assets_analyzer.analyze(
    'calc_beta', 
    index=sp
)

#### Alpha
Alpha allows us to compare the returns of an asset to an index. We can compare to the S&P 500 using the risk-free rate of return. You can look up rates [here](https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/TextView.aspx?data=yield) or use the `StockReader.get_risk_free_rate_of_return()` method. This shows that bitcoin much higher returns than the S&P 500 (due to scale) and Apple is the top performer among the FAANG stocks: 

In [ ]:
r_f = reader.get_risk_free_rate_of_return()

all_assets_analyzer.analyze(
    'calc_alpha', 
    index=sp, 
    r_f=r_f
)


#### Cumulative Returns
Cumulative returns show us what we would have made for a dollar investment at time period 0. This can be multiplied by the actual investment to get the return:

In [ ]:
from cycler import cycler

# To make this a little easier to see in the black and white text

colors = list(
    plt.get_cmap('tab10')(x/10) 
    for x in range(10)
)
linestyles = ['dashed', 'solid', 'dashdot', 'dotted', 'solid']

bw_viz_cycler = (
    cycler(color=colors) + cycler(linestyle=linestyles*2)
)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].set_prop_cycle(bw_viz_cycler)

cumulative_returns = all_assets_analyzer.analyze('calc_cumulative_returns')

for name, data in cumulative_returns.items():
    data.plot(
        ax=axes[1] if name == 'Bitcoin' else axes[0], 
        label=name,
        legend=True
    )
fig.suptitle('Cumulative Returns')

## Modeling
For modeling tasks, we can use the `StockModeler` class:

In [ ]:
from stock_analysis import StockModeler

### Time Series Decomposition
We can decompose the Netflix time series into its trend, seasonality component, and residuals:

In [ ]:
decomposition = StockModeler.decompose(nflx, 20) # 20 period frequency

fig = decomposition.plot()
fig.suptitle('Netflix Stock Price Time Series Decomposition', y=1)
fig.set_figheight(6)
fig.set_figwidth(10)
fig.tight_layout()


### ARIMA
We can use ARIMA to model the performance with autoregressive (AR), differences or lagged data (I), and moving average (MA) terms. The autocorrelation plot can help find a good starting point for this:

In [ ]:
# We can use autocorrelation plots to find a good starting point for ARIMA

from pandas.plotting import autocorrelation_plot
autocorrelation_plot(nflx.close)

Lag plots can also be helpful:

In [ ]:
from pandas.plotting import lag_plot
lag_plot(nflx.close)

ARIMA takes a long time to train, so we will start with an AR of 10:

In [ ]:
%%capture
# This takes a long time to run, so we will start with a smaller AR of 10

arima_model = StockModeler.arima(nflx, ar=10, i=1, ma=5)

The `summary()` method shows statistics on the model's performance:

In [ ]:
# AR = 10, I = 1, MA = 5
print(arima_model.summary())

The residuals appear to be normally distributed (right subplot); however, their size appears to depend on time (left subplot), so this may not be a good model:

In [ ]:
StockModeler.plot_residuals(arima_model)

### Linear Regression
We can build a linear regression using the last value to determine the next value:

In [ ]:
X, Y, lm = StockModeler.regression(nflx)
print(lm.summary())

Once again, the residuals appear to depend on time:

In [ ]:
StockModeler.plot_residuals(lm)

### Compare Models Against Actuals
Let's see how these models compared to what actually happened:

In [ ]:
import datetime as dt

start = dt.date(2021, 1, 1)
end = dt.date(2021, 1, 14)

jan = stock_analysis\
    .StockReader(start, end)\
    .get_ticker_data('NFLX')

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

arima_ax = StockModeler.arima_predictions(
    nflx, 
    arima_model, 
    start=start, 
    end=end,
    ax=axes[0], 
    title='ARIMA', 
    color='b'
)

jan.close.plot(
    ax=arima_ax, 
    style='b--', 
    label='Actual Close'
)
arima_ax.legend()
arima_ax.set_ylabel('price ($)')

linear_reg = StockModeler.regression_predictions(
    nflx, 
    lm, 
    start=start, 
    end=end,
    ax=axes[1], 
    title='Linear Regression', 
    color='b'
)
jan.close.plot(
    ax=linear_reg, 
    style='b--', 
    label='Actual Close')
linear_reg.legend()
linear_reg.set_ylabel('price ($)')
